In [20]:
import pandas as pd
import numpy as np
import string
import re
from collections import Counter
import seaborn as sns
import matplotlib.pyplot as plt
import janitor 

In [53]:
data_2020 = pd.read_csv("311_data/311_2020.csv")
data_2021 = pd.read_csv("311_data/311_2021.csv")
data_2022 = pd.read_csv("311_data/311_2022.csv")
data_2023 = pd.read_csv("311_data/311_2023.csv")
data_2024 = pd.read_csv("311_data/311_2024.csv")

In [26]:
datasets = [data_2020 , data_2021, data_2022, data_2023, data_2024]
for data in datasets:
    data = data.clean_names()

In [ ]:
# # Code copied from Ray's Feasability
# Create a dictionary to store all the data
data_dict = {
    2020: data_2020,
    2021: data_2021,
    2022: data_2022,
    2023: data_2023,
    2024: data_2024
}

for year, df in data_dict.items():
    print(f"data_{year}: {df.shape}")

# Check the consistency of the columns
for year, df in data_dict.items():
    print(f"{year}: {df.columns}")

data_2020: (26780, 28)
data_2021: (35644, 26)
data_2022: (54382, 26)
data_2023: (64142, 26)
data_2024: (59059, 26)
2020: Index(['X', 'Y', 'FID', 'service_re', 'service__1', 'sap_notifi',
       'date_requested', 'case_age_d', 'case_recor', 'service_na',
       'service__2', 'date_closed', 'status', 'lat', 'lng', 'street_add',
       'zipcode', 'council_di', 'comm_plan_', 'comm_plan1', 'park_name',
       'case_origi', 'specify_th', 'referred', 'iamfloc', 'floc', 'public_url',
       'public_description'],
      dtype='object')
2021: Index(['x', 'y', 'service_request_id', 'service_request_parent_id',
       'sap_notification_number', 'date_requested', 'case_age_days',
       'case_record_type', 'service_name', 'service_name_detail',
       'date_closed', 'status', 'lat', 'lng', 'street_address', 'zipcode',
       'council_district', 'comm_plan_code', 'comm_plan_name', 'park_name',
       'case_origin', 'referred', 'iamfloc', 'floc', 'public_description',
       'ObjectId'],
      dtype=

### Standardize the column names

In [77]:
# Rename some columns in data_2020
data_2020_clean = data_2020.drop(["service__1", "service__2", "FID", "public_url", "specify_th"], axis=1)
data_2020_clean.rename(columns={"X": "x",
                          "Y": "y",
                          "service_re": "service_request_id",
                          "sap_notifi": "sap_notification_number",
                          "case_age_d": "case_age_days",
                          "case_recor": "case_record_type",
                          "service_na": "service_name",
                          "street_add": "street_address",
                          "council_di": "council_district",
                          "comm_plan_": "comm_plan_code",
                          "comm_plan1": "comm_plan_name",
                          "case_origi": "case_origin",
                          "public_des": "public_description",
                          "date_reque": "date_requested",
                          "date_close": "date_closed",

                          }, inplace=True)





In [78]:
# Only select the new 2020 columns from the other years
#print(data_2020_clean.columns)

data_2021_clean = data_2021.reindex(columns = data_2020_clean.columns)
#print(data_2021_clean.columns)
data_2022_clean = data_2022.reindex(columns = data_2020_clean.columns)
data_2023_clean = data_2023.reindex(columns = data_2020_clean.columns)
data_2024_clean = data_2024.reindex(columns = data_2020_clean.columns)

Concatenate the 311 datasets by row (stacked on top of each other)

In [79]:
data_concat = pd.concat([data_2020_clean, data_2021_clean, data_2022_clean, data_2023_clean, data_2024_clean])

In [80]:
data_concat.head()

,x,y,service_request_id,sap_notification_number,date_requested,case_age_days,case_record_type,service_name,date_closed,status,...,zipcode,council_district,comm_plan_code,comm_plan_name,park_name,case_origin,referred,iamfloc,floc,public_description
0,-1.303485e+07,3.854326e+06,2856645.0,,2020/01/19 11:43:00,2.0,Neighborhood Policing,Encampment,2020/01/21 00:00:00+00,Closed,...,92113.0,4.0,11.0,Encanto Neighborhoods,,Mobile,,SS-029551,SS-029551,Trash left
1,-1.304207e+07,3.860746e+06,2856658.0,,2020/01/19 12:07:00,2.0,Neighborhood Policing,Encampment,2020/01/21 00:00:00+00,Closed,...,92103.0,3.0,1.0,Balboa Park,BALBOA PARK,Mobile,,,,Homeless people smoking crack next to playgrou...
2,-1.303384e+07,3.862849e+06,2856660.0,,2020/01/19 12:08:00,2.0,Neighborhood Policing,Encampment,2020/01/21 00:00:00+00,Closed,...,92115.0,9.0,56.0,Mid-City:City Heights,,Mobile,,SS-026877,SS-026877,"People outside back gate, confrontational, smo..."
3,-1.305164e+07,3.860843e+06,2856674.0,,2020/01/19 12:37:00,0.0,Neighborhood Policing,Encampment,2020/01/19 00:00:00+00,Closed,...,92107.0,2.0,30.0,Peninsula,,Web,,SS-019655,SS-019655,Homeless person in front yard of recently sold...
4,-1.304231e+07,3.862072e+06,2856676.0,,2020/01/19 12:39:00,5.0,ESD Complaint/Report,Encampment,2020/01/24 00:00:00+00,Closed,...,92103.0,3.0,42.0,Uptown,,Mobile,,,,This alley has become a regular living area fo...


Create an variable on the proportion of letters that are all caps (besides the start of a sentence)

In [85]:
def remove_sentence_starts(text):
    if pd.isna(text):
        return ""
    return re.sub(r'(^[A-Z])|(?<=[.!?]\s)[A-Z]', '', text)

no_sentence_starts = data_concat["public_description"].apply(remove_sentence_starts)
data_concat["prop_uppercase"] = (no_sentence_starts.str.count(r"[A-Z]"))/(no_sentence_starts.str.len())

In [86]:
data_concat["prop_uppercase"]

0        0.000000
1        0.000000
2        0.000000
3        0.000000
4        0.000000
           ...   
59054         NaN
59055    0.015306
59056    0.000000
59057    0.000000
59058    0.000000
Name: prop_uppercase, Length: 231143, dtype: float64

In [87]:
data_concat[["public_description","prop_uppercase"]]

,public_description,prop_uppercase
0,Trash left,0.000000
1,Homeless people smoking crack next to playgrou...,0.000000
2,"People outside back gate, confrontational, smo...",0.000000
3,Homeless person in front yard of recently sold...,0.000000
4,This alley has become a regular living area fo...,0.000000
...,...,...
59054,NaN,NaN
59055,Homeless people have taking to outright squatt...,0.015306
59056,There is a gate open ad a transient camp on pr...,0.000000
59057,Multiple homeless camped on sidewalk. Please c...,0.000000


In [89]:
# From Ray's Feasability
def clean_text(df, col_name):
    return (
        df[col_name]
        .fillna("")     #Replaces all missing values with an empty string ""
        .str.lower()    #Converts all text to lowercase
        .str.replace(f"[{string.punctuation}]", "", regex=True) #Removes all punctuation marks from the text
        .str.replace(r"\s+", " ", regex=True) #Replaces multiple spaces or whitespace characters
        .str.strip()    #Removes leading and trailing spaces from each text string
    )

data_concat["clean_text"] = clean_text(data_concat, "public_description")
data_concat["word_count"] = data_concat["clean_text"].str.split().str.len()

In [90]:
data_concat.head()

,x,y,service_request_id,sap_notification_number,date_requested,case_age_days,case_record_type,service_name,date_closed,status,...,comm_plan_name,park_name,case_origin,referred,iamfloc,floc,public_description,prop_uppercase,clean_text,word_count
0,-1.303485e+07,3.854326e+06,2856645.0,,2020/01/19 11:43:00,2.0,Neighborhood Policing,Encampment,2020/01/21 00:00:00+00,Closed,...,Encanto Neighborhoods,,Mobile,,SS-029551,SS-029551,Trash left,0.0,trash left,2
1,-1.304207e+07,3.860746e+06,2856658.0,,2020/01/19 12:07:00,2.0,Neighborhood Policing,Encampment,2020/01/21 00:00:00+00,Closed,...,Balboa Park,BALBOA PARK,Mobile,,,,Homeless people smoking crack next to playgrou...,0.0,homeless people smoking crack next to playgrou...,16
2,-1.303384e+07,3.862849e+06,2856660.0,,2020/01/19 12:08:00,2.0,Neighborhood Policing,Encampment,2020/01/21 00:00:00+00,Closed,...,Mid-City:City Heights,,Mobile,,SS-026877,SS-026877,"People outside back gate, confrontational, smo...",0.0,people outside back gate confrontational smoki...,16
3,-1.305164e+07,3.860843e+06,2856674.0,,2020/01/19 12:37:00,0.0,Neighborhood Policing,Encampment,2020/01/19 00:00:00+00,Closed,...,Peninsula,,Web,,SS-019655,SS-019655,Homeless person in front yard of recently sold...,0.0,homeless person in front yard of recently sold...,9
4,-1.304231e+07,3.862072e+06,2856676.0,,2020/01/19 12:39:00,5.0,ESD Complaint/Report,Encampment,2020/01/24 00:00:00+00,Closed,...,Uptown,,Mobile,,,,This alley has become a regular living area fo...,0.0,this alley has become a regular living area fo...,43


In [91]:
data_concat.to_csv("all_years_311.csv", index=False)